In [1]:
import pprint
import urllib.parse
import json5
from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool

### 自定义天气查询工具

In [2]:
import os
import requests
@register_tool('MyWeather')
class MyWeather(BaseTool):
    description = "获取天气信息"
    parameters = [{
        'name': 'prompt',
        'type': 'string',
        'description': '城市/区具体名称，如`北京市海淀区`请描述为`海淀区`',
        'required': True
    }]
    def __init__(self,cfg):
        super().__init__()
        self.url = 'https://restapi.amap.com/v3/weather/weatherInfo?city={city}&key={key}'

        import pandas as pd
        self.city_df = pd.read_excel(
            'https://modelscope.oss-cn-beijing.aliyuncs.com/resource/agent/AMap_adcode_citycode.xlsx')
        self.token = os.environ.get('AMAP_TOKEN', '')

    def get_city_adcode(self, city_name):
        filtered_df = self.city_df[self.city_df['中文名'] == city_name]
        if len(filtered_df['adcode'].values) == 0:
            raise ValueError(f'location {city_name} not found, availables are {self.city_df["中文名"]}')
        else:
            return filtered_df['adcode'].values[0]
        pass
    
    def call(self, params) -> str:
        params = self._verify_json_format_args(params)

        location = params['location']
        print("location:",location)
        response = requests.get(self.url.format(city=self.get_city_adcode(location), key=self.token))
        data = response.json()
        if data['status'] == '0':
            raise RuntimeError(data)
        else:
            weather = data['lives'][0]['weather']
            temperature = data['lives'][0]['temperature']
            return f'{location}的天气是{weather}温度是{temperature}度。'


### 配置使用LLM内容

In [3]:
llm_cfg = {
    # 使用 DashScope 提供的模型服务：
    # 'model': 'qwen-max',
    # 'model_server': 'dashscope',
    # 'api_key': 'YOUR_DASHSCOPE_API_KEY',
    # 如果这里没有设置 'api_key'，它将读取 `DASHSCOPE_API_KEY` 环境变量。

    # 使用与 OpenAI API 兼容的模型服务，例如 vLLM 或 Ollama：
    'model': '/models/chatglm3-6b',
    'model_server': 'http://172.16.7.132:23148/v1',  # base_url，也称为 api_base
    # 'api_key': 'EMPTY',
    # "AMAP_TOKEN":""

    # （可选） LLM 的超参数：
    # 'generate_cfg': {
    #     'top_p': 0.8
    # }
}

### 创建智能体

In [4]:
system_instruction = '''你是一个乐于助人的AI助手。
在收到用户的请求后，你应该：
- 判断用户是否需要查询天气信息
- 如果需要,返回查询到天气信息
- 否则礼貌结束即可
'''

tools = ["MyWeather",]
bot = Assistant(llm=llm_cfg,
                system_message=system_instruction,
                function_list=tools,
                files=None)


In [5]:
query="深圳天气怎么样"
messages=[{"role":"user","content":query}]
for response in bot.run(messages=messages):
    print(response)

[{'role': 'assistant', 'content': ' 好的'}]
[{'role': 'assistant', 'content': ' 好的，'}]
[{'role': 'assistant', 'content': ' 好的，我可以'}]
[{'role': 'assistant', 'content': ' 好的，我可以帮你'}]
[{'role': 'assistant', 'content': ' 好的，我可以帮你查询'}]
[{'role': 'assistant', 'content': ' 好的，我可以帮你查询深圳'}]
[{'role': 'assistant', 'content': ' 好的，我可以帮你查询深圳的'}]
[{'role': 'assistant', 'content': ' 好的，我可以帮你查询深圳的天气'}]
[{'role': 'assistant', 'content': ' 好的，我可以帮你查询深圳的天气。'}]
[{'role': 'assistant', 'content': ' 好的，我可以帮你查询深圳的天气。请'}]
[{'role': 'assistant', 'content': ' 好的，我可以帮你查询深圳的天气。请告诉我'}]
[{'role': 'assistant', 'content': ' 好的，我可以帮你查询深圳的天气。请告诉我深圳'}]
[{'role': 'assistant', 'content': ' 好的，我可以帮你查询深圳的天气。请告诉我深圳的具体'}]
[{'role': 'assistant', 'content': ' 好的，我可以帮你查询深圳的天气。请告诉我深圳的具体位置'}]
[{'role': 'assistant', 'content': ' 好的，我可以帮你查询深圳的天气。请告诉我深圳的具体位置，'}]
[{'role': 'assistant', 'content': ' 好的，我可以帮你查询深圳的天气。请告诉我深圳的具体位置，如'}]
[{'role': 'assistant', 'content': ' 好的，我可以帮你查询深圳的天气。请告诉我深圳的具体位置，如`'}]
[{'role': 'assistant', 'content': ' 好